# Continue analysis on Twitter Network: Wordclouds

In [3]:
import wordcloud
import matplotlib.pyplot as plt
import pandas as pd

### Import tweets from NLP analysis:

In [6]:
tweets_analysis = pd.read_csv('tweets_analysis.csv')
tweets_analysis.head()
tweets_analysis.drop('Unnamed: 0', axis=1, inplace=True)

#### Can I make a word cloud from my saved tweets, or do I have to use the Twitter API again and search by words and geocode?

In [ ]:
wordcloud = wordcloud.WordCloud(max_font_size = 150, max_words = 100, backgroun_color='white').generate(str())